In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import urllib.request
import time
import csv
import os
import pandas as pd
import urllib.request

In [3]:
#이미지 다운로드 폴더 생성
if not os.path.isdir("img"):
    os.makedirs("img")
# 웹드라이버 실행
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\Administrator\AppData\Local\Temp\ipykernel_16692\3509940763.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
# 영화순위 년도 설정
pageList = [2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012]

# 영화코드 크롤링
codeList = []
for page in pageList :
    url = f'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&qvt=0&query={page}%EB%85%84%20%EC%98%81%ED%99%94%20%EC%88%9C%EC%9C%84'
    driver.get(url)
    time.sleep(0.3)
    codes = driver.find_elements(By.CSS_SELECTOR, '.movie_info > .btn_area > .btn_down')
    rank = 1
    for code in codes : 
        codeList.append(code.get_attribute('href')[-6:].replace("=", ""))
        rank += 1
        if rank == 61 :
            break

# CSS선택자가 다른 영화코드 삭제
codeList.remove('204640')

print(len(codeList))

660


In [7]:
# 컬럼 생성
titleList = []
ratingList = []
typeList = []
contentList = []
openDateList = []
runtimeList = []
directorList = []
actorList = []
countryList = []
imgPathList = []

# 영화코드 페이지별 크롤링
for code in codeList :
    url = f'https://movie.naver.com/movie/bi/mi/basic.naver?code={code}'
    driver.get(url)
    time.sleep(0.8)
    types = driver.find_elements(By.CSS_SELECTOR, 'dd > p > span')
    actors = driver.find_elements(By.CSS_SELECTOR, 'dd > p')
    title = driver.find_elements(By.CSS_SELECTOR, '.h_movie')[1].text.replace("\n상영중", "")

    # 결측치 컬럼 빈칸으로 대체
    try :
        openDateList.append(types[3].text.replace(" ", "")[0:10])
        directorList.append(actors[1].text)
        actorList.append(actors[2].text)
        #이미지 다운로드
        path = driver.find_element(By.CSS_SELECTOR, '.poster > a > img').get_attribute('src').split('?')[0]
        urllib.request.urlretrieve(path, "img/" + title.replace(":", "") + ".jpg")
        imgPathList.append(title.replace(":", "") + ".jpg")
        contentList.append(driver.find_element(By.CSS_SELECTOR, '.con_tx').text)
    except :
        openDateList.append("")
        directorList.append("")
        actorList.append("")
        imgPathList.append("")
        contentList.append("")

    titleList.append(title)
    ratingList.append(driver.find_elements(By.CSS_SELECTOR, '.sc_view > .star_score > em')[0].text)
    typeList.append(types[0].text)
    runtimeList.append(types[2].text)
    countryList.append(types[1].text)

    # 크롤링 데이터 확인
    print(code)
    print(len(titleList), len(typeList), len(contentList), len(imgPathList), len(openDateList),
    len(ratingList), len(runtimeList), len(actorList), len(directorList), len(countryList))

192608
1 1 1 1 1 1 1 1 1 1
81888
2 2 2 2 2 2 2 2 2 2
201641
3 3 3 3 3 3 3 3 3 3
182016
4 4 4 4 4 4 4 4 4 4
195758
5 5 5 5 5 5 5 5 5 5
191646
6 6 6 6 6 6 6 6 6 6
196367
7 7 7 7 7 7 7 7 7 7
187347
8 8 8 8 8 8 8 8 8 8
191634
9 9 9 9 9 9 9 9 9 9
198413
10 10 10 10 10 10 10 10 10 10
192151
11 11 11 11 11 11 11 11 11 11
196854
12 12 12 12 12 12 12 12 12 12
164122
13 13 13 13 13 13 13 13 13 13
187821
14 14 14 14 14 14 14 14 14 14
154282
15 15 15 15 15 15 15 15 15 15
210852
16 16 16 16 16 16 16 16 16 16
187320
17 17 17 17 17 17 17 17 17 17
219812
18 18 18 18 18 18 18 18 18 18
177366
19 19 19 19 19 19 19 19 19 19
190694
20 20 20 20 20 20 20 20 20 20
191430
21 21 21 21 21 21 21 21 21 21
80737
22 22 22 22 22 22 22 22 22 22
190400
23 23 23 23 23 23 23 23 23 23
213733
24 24 24 24 24 24 24 24 24 24
50368
25 25 25 25 25 25 25 25 25 25
190991
26 26 26 26 26 26 26 26 26 26
217719
27 27 27 27 27 27 27 27 27 27
190695
28 28 28 28 28 28 28 28 28 28
211161
29 29 29 29 29 29 29 29 29 29
213481
30 30 30 30 3

In [8]:
# 데이터프레임 생성 및 저장
movie_dic = {
            'movie_title' : titleList,
            'movie_type' : typeList,
            'movie_content' : contentList,
            'movie_img' : imgPathList,
            'movie_open_date' : openDateList,
            'movie_rating' : ratingList,
            'movie_runtime' : runtimeList,
            'movie_actor' : actorList,
            'movie_director' : directorList,
            'movie_country' : countryList
            }

movie_df = pd.DataFrame(movie_dic)
movie_df.to_csv("movie.csv", encoding='utf-8')
movie_df


,movie_title,movie_type,movie_content,movie_img,movie_open_date,movie_rating,movie_runtime,movie_actor,movie_director,movie_country
0,범죄도시2,"범죄, 액션","가리봉동 소탕작전 후 4년 뒤,\n 금천서 강력반은 베트남으로 도주한 용의자를 인도...",범죄도시2.jpg,2022.05.18,9.31,106분,"마동석(마석도), 손석구(강해상), 최귀화(전일만)",이상용,대한민국
1,탑건: 매버릭,액션,최고의 파일럿이자 전설적인 인물 매버릭(톰 크루즈)은 자신이 졸업한 훈련학교 교관으...,탑건 매버릭.jpg,2022.06.22,9.76,130분,"톰 크루즈(매버릭), 마일즈 텔러(루스터), 제니퍼 코넬리(페니)",조셉 코신스키,미국
2,공조2: 인터내셔날,"액션, 코미디",공조 이즈 백! 이번엔 삼각 공조다!\n \n 남한으로 숨어든 글로벌 범죄 조직을 ...,공조2 인터내셔날.jpg,2022.09.07,8.44,129분,"현빈(림철령), 유해진(강진태), 윤아(박민영)",이석훈,대한민국
3,닥터 스트레인지: 대혼돈의 멀티버스,"액션, 판타지, 모험","끝없이 균열되는 차원과 뒤엉킨 시공간의 멀티버스가 열리며\n 오랜 동료들, 그리고 ...",닥터 스트레인지 대혼돈의 멀티버스.jpg,2022.05.04,7.22,126분,"베네딕트 컴버배치(닥터 스트레인지), 엘리자베스 올슨(완다 막시모프 / 스칼렛 위치)",샘 레이미,미국
4,헌트,"액션, 드라마","[조직 내 숨어든 스파이를 색출하라!\n ‘사냥꾼’이 될 것인가, ‘사냥감’이 될 ...",헌트.jpg,2022.08.10,8.37,125분,"이정재(박평호), 정우성(김정도), 전혜진(방주경)",이정재,대한민국
...,...,...,...,...,...,...,...,...,...,...
654,점쟁이들,"코미디, 공포","점쟁이들의 리더이자 귀신 쫓는 점쟁이 박선생(김수로), 공학박사 출신의 과학 하는 ...",점쟁이들.jpg,2012.10.03,5.84,119분,"김수로(박선생), 강예원(찬영), 이제훈(석현)",신정원,대한민국
655,주먹왕 랄프,"애니메이션, 코미디, 가족",8비트 게임 ‘다고쳐 펠릭스’에서 건물을 부수는 악당 주먹왕 랄프. 30년째 매일같...,주먹왕 랄프.jpg,2012.12.19,9.35,108분,"존 C. 라일리(랄프 목소리), 사라 실버맨(바넬로피 목소리), 잭 맥브레이어(펠릭...",리치 무어,미국
656,타이탄의 분노,"액션, 판타지",타이탄에 맞서라! 제우스를 구하라!\n 크라켄과의 전투를 승리로 이끈 반신반인 ‘페...,타이탄의 분노.jpg,2012.03.29,6.86,99분,"샘 워싱턴(페르세우스), 리암 니슨(제우스), 레이프 파인즈(하데스)",조나단 리브스만,미국
657,디스 민즈 워,"액션, 멜로/로맨스, 코미디",세계 최고의 실력을 가진 CIA 특수요원 터크(톰 하디)와 프랭클린(크리스 파인)....,디스 민즈 워.jpg,2012.02.29,8.20,97분,"리즈 위더스푼(로렌), 크리스 파인(프랭클린(FDR)), 톰 하디(터크)",맥지,미국


In [9]:
# 결측치 데이터 확인
df = pd.read_csv('movie.csv', index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 659 entries, 0 to 658
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_title      659 non-null    object 
 1   movie_type       659 non-null    object 
 2   movie_content    656 non-null    object 
 3   movie_img        656 non-null    object 
 4   movie_open_date  656 non-null    object 
 5   movie_rating     659 non-null    float64
 6   movie_runtime    659 non-null    object 
 7   movie_actor      656 non-null    object 
 8   movie_director   656 non-null    object 
 9   movie_country    659 non-null    object 
dtypes: float64(1), object(9)
memory usage: 56.6+ KB


In [10]:
# 결측치 데이터 확인 결측치 제거
df.dropna(axis=0, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 656 entries, 0 to 658
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_title      656 non-null    object 
 1   movie_type       656 non-null    object 
 2   movie_content    656 non-null    object 
 3   movie_img        656 non-null    object 
 4   movie_open_date  656 non-null    object 
 5   movie_rating     656 non-null    float64
 6   movie_runtime    656 non-null    object 
 7   movie_actor      656 non-null    object 
 8   movie_director   656 non-null    object 
 9   movie_country    656 non-null    object 
dtypes: float64(1), object(9)
memory usage: 56.4+ KB


In [ ]:
# actor 이상값 색출
movie_actor = df["movie_actor"].to_list()
for i in range(len(movie_actor)) :    
    if "[국내] 12세 관람가" == movie_actor[i] :
        print(i)
    elif "[국내] 15세 관람가" == movie_actor[i] :
        print(i)
    elif "청소년 관람불가" == movie_actor[i] :
        print(i)
    elif "[국내] 전체 관람가" == movie_actor[i] :
        print(i)

In [ ]:
# 이상컬럼 삭제
df = df.drop(index=17)
df = df.drop(index=26)
df = df.drop(index=36)
df = df.drop(index=40)
df = df.drop(index=45)
df = df.drop(index=98)

In [ ]:
# movie_seq 컬럼 추가
movie_actor = df["movie_actor"].to_list()
movie_seq = []

for i in range(1, len(movie_actor)+1) :
    movie_seq.append(i)
len(movie_seq)

df["movie_seq"] = movie_seq

In [11]:
# 전처리 데이터 저장
df.to_csv("moviedata_df.csv", encoding='utf-8')